In [1]:
import torch
from transformers import AutoModelForCausalLM
import peft 
from peft import LoraConfig
from peft import get_peft_model

/opt/anaconda3/envs/aac-story/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [3]:
model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-2-1124-7B")

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/355 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [4]:
model = get_peft_model(model, config)

In [5]:
model.print_trainable_parameters()

trainable params: 39,976,960 || all params: 7,338,594,304 || trainable%: 0.5447495575305208


# TODO - review documentation to make sure this is complete
https://huggingface.co/docs/peft/main/en/package_reference/config#peft.PeftConfig
